<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab4/lab5_Transformer_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab4

Mounted at /content/drive
/content/drive/MyDrive/ai-intensive2
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 7.89 KiB | 33.00 KiB/s, done.
From https://github.com/dh610/ai-intensive2
   044141c..e77cbe2  main       -> origin/main
Updating 044141c..e77cbe2
Fast-forward
 lab4/lab5_NMT_with_Attention_assignment.ipynb | 901 ++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 901 insertions(+)
 create mode 100644 lab4/lab5_NMT_with_Attention_assignment.ipynb
/content/drive/MyDrive/ai-intensive2/lab4


# Lab 5 : Neural Machine Translation with Attention Mechanism

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.

---


## 목차

1. Transformer
2. Multi-Head Attention
3. Position-wise Feed-Forward Networks
4. Positional Encoding
5. Encoder / Decoder
6. Transformer Model
7. Training and Evaluation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

/home/iknow/anaconda3/envs/cpwan/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Transformer

- [Attention is All You Need](https://arxiv.org/abs/1706.03762) 논문을 바탕으로 제안된 모델
- 기존의 RNN, LSTM, GRU 등의 모델을 사용하지 않고, Attention Mechanism만을 사용하여 번역을 수행하는 모델

## 2. Multi-Head Attention

- Multi-Head Attention mechanism은 seqeuence의 각 position pari에 대해 attention을 각각 수행하는 것을 의미

![multi-head-attention](./data/multi-head-attn.png)

- [BLANK]는 input seqeucen의 각기 다른 aspect에 대해 학습하도록 함

- Multi-Head Attention은 다음과 같이 수행
    - $MultiHead(Q, K, V) = Concat(head_1, ..., head_h)W^O$
    - $head_i = Attention(QW_i^Q, KW_i^K, VW_i^V)$


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

## 3. Position-wise Feed-Forward Networks

- Position-wise Feed-Forward Networks는 각 position에 독립적으로 적용되는 fully-connected feed-forward network

- $FFN(x) = max(0, xW_1 + b_1)W_2 + b_2$

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        """ Fill this cell to implement Position-Wise Feed Forward Layer
        self.fc1 = nn.Linear(
        self.fc2 = nn.Linear(
        self.relu = nn.ReLU()

        """

    def forward(self, x):
        """ Define forward pass of Position-Wise Feed Forward Layer
        x = self.fc1(x)
        x =
        ...

        """

        return x

## 4. Positional Encoding

- [BLANK]은 input sequence의 각 position에 대해 embedding vector를 더해줌으로써 position 정보를 추가하는 방법

- 일반적인 seq2seq 모델과는 다르게 Transformer는 input sequence를 한 번에 받아 처리하기 때문에 position 정보를 추가해주어야 함

- Positional Encoding은 다음과 같이 수행
    - $PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})$
    - $PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})$

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

## 5. Encoder / Decoder

### Encoder

- Encoder는 다음과 같이 구성
    - Input Embedding
    - Positional Encoding
    - N개의 Encoder Layer
        - Multi-Head Attention
        - Add & Norm
        - Position-wise Feed-Forward Networks
        - Add & Norm

[ANSWER] : 각 module이 어떤 역할을 하는지 서술하시오

![Encoder](./data/transformer-encoder-layer.png)


In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        """ Fill this cell to implement Transformer Encoder Layer
        self.self_attn = MultiHeadAttention(
        self.feed_forward = PositionWiseFeedForward(
        self.norm1 = nn.LayerNorm(
        self.norm2 = nn.LayerNorm(
        self.dropout = nn.Dropout(

        """

    def forward(self, x, mask):
        """ Define forward pass of Transformer Encoder Layer
        attn_output = self.self_attn(
        x = self.norm1(
        ff_output = self.feed_forward(
        x = self.norm2(
        """
        return x

### Decoder

- Decoder는 다음과 같이 구성
    - Target Embedding
    - Positional Encoding
    - N개의 Decoder Layer
        - Masked Multi-Head Attention
        - Add & Norm
        - Multi-Head Attention
        - Add & Norm
        - Position-wise Feed-Forward Networks
        - Add & Norm
    - Linear & Softmax

[ANSWER] : 각 module이 어떤 역할을 하는지 서술하시오

![Decoder](./data/transformer-decoder-layer.png)

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        """ Fill this cell to implement Transformer Decoder Layer
        self.self_attn = MultiHeadAttention(
        self.cross_attn = MultiHeadAttention(
        self.feed_forward = PositionWiseFeedForward(
        self.norm1 = nn.LayerNorm(
        self.norm2 = nn.LayerNorm(
        self.norm3 = nn.LayerNorm(
        self.dropout = nn.Dropout(

        """

    def forward(self, x, enc_output, src_mask, tgt_mask):
        """ Fill this cell to implement forward pass of Transformer Decoder Layer
        attn_output = self.self_attn(
        x = self.norm1(
        attn_output = self.cross_attn(
        x = self.norm2(
        ff_output = self.feed_forward(
        x = self.norm3(
        """

        return x


### Transformer

- Merge Encoder & Decoder into Transformer

![Transformer](./data/transformer.png)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [ ]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)


In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.695968627929688
Epoch: 2, Loss: 8.5587739944458
Epoch: 3, Loss: 8.48740005493164
Epoch: 4, Loss: 8.427062034606934
Epoch: 5, Loss: 8.375468254089355
Epoch: 6, Loss: 8.307201385498047
Epoch: 7, Loss: 8.233159065246582
Epoch: 8, Loss: 8.148744583129883
Epoch: 9, Loss: 8.068256378173828
Epoch: 10, Loss: 7.987551689147949
Epoch: 11, Loss: 7.906238079071045
Epoch: 12, Loss: 7.825662136077881
Epoch: 13, Loss: 7.738952159881592
Epoch: 14, Loss: 7.658864498138428
Epoch: 15, Loss: 7.574713230133057
Epoch: 16, Loss: 7.498396396636963
Epoch: 17, Loss: 7.411505699157715
Epoch: 18, Loss: 7.326872825622559
Epoch: 19, Loss: 7.244997024536133
Epoch: 20, Loss: 7.169650554656982
Epoch: 21, Loss: 7.084219455718994
Epoch: 22, Loss: 7.005242824554443
Epoch: 23, Loss: 6.929657459259033
Epoch: 24, Loss: 6.853853702545166
Epoch: 25, Loss: 6.774919509887695
Epoch: 26, Loss: 6.708837032318115
Epoch: 27, Loss: 6.6422038078308105
Epoch: 28, Loss: 6.561357498168945
Epoch: 29, Loss: 6.495044231414